### Effet du support

Ce graphique interactif illustre l’effet du changement de support sur la distribution des teneurs.

Au départ, deux gisements sont simulés avec la **même distribution statistique** à l’échelle du **support ponctuel (1 m × 1 m)**.  
Pour observer l’impact de l’agrandissement du support, vous pouvez utiliser le **widget interactif ci-dessous** afin de modifier la taille du bloc de support.

Par exemple, la teneur d’un bloc de **10 m × 10 m** est calculée comme la **moyenne des teneurs** des cellules de **1 m × 1 m** qu’il contient.

Les **cartes**, les **histogrammes** et les **fonctions de répartition cumulées** sont mis à jour dynamiquement à chaque changement de support. Cela vous permet de visualiser concrètement l’**effet de l’agrégation spatiale** sur les propriétés statistiques du gisement.

**Amusez-vous à explorer les résultats — nous en discuterons en classe !**


In [24]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from numpy.fft import fft2, ifft2, fftshift

# --- Covariance sphérique (FFT)
def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

# --- FFT-MA avec recadrage à N×N
def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

# --- Transformation lognormale
def gaussian_to_lognormal(field):
    return np.exp(field)

# --- Apparier les histogrammes
def match_histogram(reference, target):
    flat_ref = np.sort(reference.ravel())
    sorted_idx = np.argsort(target.ravel())
    result = np.zeros_like(target.ravel())
    result[sorted_idx] = flat_ref
    return result.reshape(target.shape)

# --- Agrégation (effet de support)
def aggregate(field, block_size):
    s = field.shape[0]
    reduced_size = s // block_size
    aggregated = np.zeros((reduced_size, reduced_size))
    for i in range(reduced_size):
        for j in range(reduced_size):
            block = field[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            aggregated[i, j] = np.mean(block)
    return aggregated

# --- Simulation initiale
size = 500
range_short = 10/size
range_long = 250/size

gauss_short = fftma_simulation(size=size, range_=range_short, seed=42)
gauss_long = fftma_simulation(size=size, range_=range_long, seed=24)

lognorm_short = gaussian_to_lognormal(gauss_short)
lognorm_long = gaussian_to_lognormal(gauss_long)
lognorm_long = match_histogram(lognorm_short, lognorm_long)

# --- Affichage interactif avec échelle fixe
def plot_fields(support):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    agg_short = aggregate(lognorm_short, support)
    agg_long = aggregate(lognorm_long, support)

    vmin, vmax = 0, 10
    bins = np.linspace(0, 10, 11)

    # Moyenne unique utilisée pour les deux visualisations
    mean = np.mean(agg_short)

    # --- Carte portée courte ---
    im0 = axes[0, 0].imshow(agg_short, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 0].set_title(f'Portée courte – Support {support}×{support}', fontsize=12)
    axes[0, 0].axis('off')
    fig.colorbar(im0, ax=axes[0, 0], fraction=0.046, pad=0.04)

    # --- Carte portée longue ---
    im1 = axes[0, 1].imshow(agg_long, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 1].set_title(f'Portée longue – Support {support}×{support}', fontsize=12)
    axes[0, 1].axis('off')
    fig.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)

    # --- Histogrammes superposés ---
    axes[1, 0].hist(agg_short.ravel(), bins=bins, alpha=0.6, color='steelblue', label='Portée courte', edgecolor='black')
    axes[1, 0].hist(agg_long.ravel(), bins=bins, alpha=0.6, color='darkorange', label='Portée longue', edgecolor='black')
    axes[1, 0].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')
    axes[1, 0].set_title('Histogrammes superposés', fontsize=12)
    axes[1, 0].set_xlabel('Teneur')
    axes[1, 0].set_ylabel('Fréquence')
    axes[1, 0].set_xlim([0, 10])
    axes[1, 0].grid(True)
    axes[1, 0].legend()

    # --- ECDF superposées ---
    for data, color, label in zip(
        [agg_short.ravel(), agg_long.ravel()],
        ['steelblue', 'darkorange'],
        ['Portée courte', 'Portée longue']
    ):
        sorted_data = np.sort(data)
        ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
        axes[1, 1].plot(sorted_data, ecdf, color=color, lw=2, label=label)

    # Lignes de moyennes en noir
    axes[1, 1].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')

    axes[1, 1].set_title('Fonctions de répartition cumulées', fontsize=12)
    axes[1, 1].set_xlabel('Teneur')
    axes[1, 1].set_ylabel('F(x)')
    axes[1, 1].set_xlim([0, 10])
    axes[1, 1].set_ylim([0, 1])
    axes[1, 1].grid(True)
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()


# --- Widget interactif
interact(plot_fields, support=IntSlider(min=1, max=50, step=2, value=1, description='Support (m)'))


interactive(children=(IntSlider(value=1, description='Support (m)', max=50, min=1, step=2), Output()), _dom_cl…

<function __main__.plot_fields(support)>

### Effet d'information

Il est essentiel de bien comprendre la notion d'information dans le contexte de l’estimation des ressources minières. En effet, plus on dispose de données fiables, plus on obtient une image précise du gisement. Mais il ne suffit pas d’avoir beaucoup de données : encore faut-il qu’elles soient de qualité, c’est-à-dire **précises** et **exactes**.

- **La précision** décrit la répétabilité des mesures. Par exemple, si l’analyse d’une même carotte donne des résultats très proches lorsqu’elle est répétée, la précision est bonne. Cela signifie que les erreurs aléatoires sont faibles.

- **L’exactitude** mesure à quel point les données sont proches de la valeur réelle. Si une méthode d’analyse ou d’estimation produit toujours une surestimation ou une sous-estimation, elle souffre d’un **biais systématique**.

Lorsque les données sont à la fois précises et exactes, on dit qu’elles sont **justes**.

Dans la pratique, les teneurs sont estimées à partir de données comme les forages et les analyses en laboratoire, mais la valeur réelle ne peut être confirmée qu’après l’extraction du minerai. De plus, on ne réalise pas 50 analyses sur la même carotte : on compare plutôt les teneurs estimées (une valeur par localisation) avec les teneurs réellement mesurées après extraction, afin d’évaluer la présence d’erreurs ou de biais.

Les écarts entre les teneurs estimées et les teneurs réelles peuvent être dus à la **variabilité des mesures** (manque de précision) ou à un **biais systématique dans la méthode d’estimation** (manque d’exactitude).

Avec ce graphique interactif, vous pouvez explorer comment la précision et l’exactitude influencent les résultats d’estimation. Amusez-vous à comparer l'effet du bruit aléatoire ou du biais systématique sur les cartes de teneur.


In [41]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift
import ipywidgets as widgets
from ipywidgets import interact

def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

def gaussian_to_lognormal(field):
    return np.exp(field)

def plot_real_vs_estimated_bias(bias_percent=10, noise_std=0.5, cutoff=2):
    size = 200
    seed = 42  # On fixe la graine pour la reproductibilité
    np.random.seed(seed)
    
    field = fftma_simulation(size=size, range_=0.2, sill=1.0, seed=seed)
    real = gaussian_to_lognormal(field)
    
    # Biais systématique multiplicatif : on ajoute X% de la valeur réelle à la valeur réelle
    biased_real = real * (1 + bias_percent / 100)
    
    # Bruit aléatoire normal centré sur la valeur réelle, écart-type noise_std
    noise = np.random.normal(loc=0, scale=noise_std, size=real.shape)
    
    # Champ estimé = valeur biaisée + bruit (le bruit peut augmenter ou diminuer)
    estimated = biased_real + noise
    
    lower_clip, upper_clip = 0, 10
    real_clipped = np.clip(real, lower_clip, upper_clip)
    estimated_clipped = np.clip(estimated, lower_clip, upper_clip)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    im0 = axes[0].imshow(real_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[0].set_title("Champ réel")
    axes[0].set_xlabel("X (m)")
    axes[0].set_ylabel("Y (m)")
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(estimated_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[1].set_title("Champ estimé")
    axes[1].set_xlabel("X (m)")
    axes[1].set_ylabel("Y (m)")
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    real_flat = real_clipped.flatten()
    estimated_flat = estimated_clipped.flatten()

    # Masques selon la description :
    # Minerai ignoré : réel ≥ cutoff ET estimé < cutoff → rouge
    mask_red = (real_flat >= cutoff) & (estimated_flat < cutoff)
    # Stérile traité : réel < cutoff ET estimé ≥ cutoff → bleu
    mask_blue = (real_flat < cutoff) & (estimated_flat >= cutoff)
    # Les autres points gris
    mask_other = ~(mask_red | mask_blue)

    # Nuage de points
    axes[2].scatter(estimated_flat[mask_other], real_flat[mask_other], alpha=0.3, s=10, color="gray", edgecolor="none", label="Correctement classé")
    axes[2].scatter(estimated_flat[mask_blue], real_flat[mask_blue], alpha=0.7, s=15, color="blue", edgecolor="k", linewidth=0.2, label="Stérile traité")
    axes[2].scatter(estimated_flat[mask_red], real_flat[mask_red], alpha=0.7, s=15, color="red", edgecolor="k", linewidth=0.2, label="Minerai ignoré")

    axes[2].plot([lower_clip, upper_clip], [lower_clip, upper_clip], 'k-', linewidth=2, label="Ligne 1:1")

    # Régression linéaire inversée : y=réel en fonction de x=estimé
    A = np.vstack([estimated_flat, np.ones_like(estimated_flat)]).T
    a, b = np.linalg.lstsq(A, real_flat, rcond=None)[0]
    x_fit = np.array([lower_clip, upper_clip])
    y_fit = a * x_fit + b
    axes[2].plot(x_fit, y_fit, 'r--', label=f"Régression linéaire\nréel={a:.2f}*estimé+{b:.2f}")

    # Lignes de coupure verticale et horizontale
    axes[2].axhline(cutoff, color='gray', linestyle='--')
    axes[2].axvline(cutoff, color='gray', linestyle='--')

    # Flèche horizontale démarrant sur la ligne verticale cutoff, au niveau cutoff en Y
    x_arrow_start = cutoff
    y_arrow = 8  # sur la ligne de coupure horizontale
    arrow_length = 2  # longueur de la flèche

    axes[2].annotate('',
                     xy=(x_arrow_start + arrow_length, y_arrow),
                     xytext=(x_arrow_start, y_arrow),
                     arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8))

    # Texte juste au-dessus de la flèche, à droite du cutoff
    text_x = x_arrow_start + arrow_length * 0.5
    text_y = y_arrow + 0.3

    axes[2].text(text_x, text_y, 'Traité', fontsize=12, ha='left', va='bottom')

    axes[2].set_xlabel("Teneur estimée (ppm)")
    axes[2].set_ylabel("Teneur réelle (ppm)")
    axes[2].set_title("Teneur réelle vs estimée")
    axes[2].set_xlim(lower_clip, upper_clip)
    axes[2].set_ylim(lower_clip, upper_clip)
    axes[2].set_aspect('equal', adjustable='box')

    # Pourcentage de points dans chaque quadrant (selon seuil cutoff)
    Q1 = np.sum((real_flat >= cutoff) & (estimated_flat >= cutoff))
    Q2 = np.sum(mask_red)   # minerai ignoré
    Q3 = np.sum((real_flat < cutoff) & (estimated_flat < cutoff))
    Q4 = np.sum(mask_blue)  # stérile traité
    total = len(real_flat)

    # Affichage des % dans les 4 coins
    axes[2].text(0.2, cutoff - 0.5, f'{Q1/total*100:.1f}%', fontsize=12)
    axes[2].text(0.2, 9.5, f'{Q2/total*100:.1f}%', fontsize=12, color='red')
    axes[2].text(cutoff + 0.2, 9.5, f'{Q3/total*100:.1f}%', fontsize=12)
    axes[2].text(8.5, 0.5, f'{Q4/total*100:.1f}%', fontsize=12, color='blue')

    axes[2].legend(loc='upper right')
    axes[2].grid(True, linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

bias_slider = widgets.FloatSlider(
    value=0,
    min=-50,
    max=50,
    step=5,
    description='Biais systématique (%)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

noise_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.5,
    step=0.01,
    description='Écart-type bruit',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

cutoff_slider = widgets.FloatSlider(
    value=2,
    min=2,
    max=8,
    step=0.1,
    description='Teneur de coupure (ppm)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

interact(plot_real_vs_estimated_bias,
         bias_percent=bias_slider,
         noise_std=noise_slider,
         cutoff=cutoff_slider);

interactive(children=(FloatSlider(value=0.0, description='Biais systématique (%)', layout=Layout(width='400px'…

### Gisement réel : Impact de la teneur de coupure, de la corrélation spatiale sur la localiation des ressources

Tout au long de la session, nous allons apprendre à choisir une teneur de coupure, à estimer la corrélation spatiale d’un gisement, et surtout à comprendre comment ces paramètres influencent l’estimation des ressources.

Mais pour l’instant… amusez-vous un peu !

Explorez ce modèle 3D interactif et observez comment la localisation des ressources évolue lorsque vous modifiez la teneur de coupure ou le degré de corrélation spatiale.
Un bon moyen de jouer les géologues tout en développant votre intuition !

**Cette image interactive est encore en développement.**



In [75]:
import numpy as np
from scipy.fftpack import fftn, ifftn
from ipywidgets import interact, FloatSlider, IntSlider
import plotly.graph_objects as go
import plotly.io as pio

def spherical_covariance_fft_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0):
    nx, ny, nz = shape
    x = np.arange(-nx, nx)
    y = np.arange(-ny, ny)
    z = np.arange(-nz, nz)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

    X_scaled = X / ax
    Y_scaled = Y / ay
    Z_scaled = Z / az

    def rotation_matrix(rx, ry, rz):
        rx = np.radians(rx)
        ry = np.radians(ry)
        rz = np.radians(rz)
        Rx = np.array([[1, 0, 0],
                       [0, np.cos(rx), -np.sin(rx)],
                       [0, np.sin(rx), np.cos(rx)]])
        Ry = np.array([[np.cos(ry), 0, np.sin(ry)],
                       [0, 1, 0],
                       [-np.sin(ry), 0, np.cos(ry)]])
        Rz = np.array([[np.cos(rz), -np.sin(rz), 0],
                       [np.sin(rz), np.cos(rz), 0],
                       [0, 0, 1]])
        return Rz @ Ry @ Rx

    R = rotation_matrix(angle_x, angle_y, angle_z)
    coords = np.stack([X_scaled, Y_scaled, Z_scaled], axis=-1)
    coords_rot = coords @ R.T

    h = np.linalg.norm(coords_rot, axis=-1)
    h = np.minimum(h, 1.0)
    cov = sill * (1 - 1.5 * h + 0.5 * h**3)
    cov[h > 1] = 0
    return cov

def fftma_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0, seed=0):
    np.random.seed(seed)
    cov = spherical_covariance_fft_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill)
    white_noise = np.random.normal(size=cov.shape)
    cov_fft = fftn(cov)
    white_fft = fftn(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    field = np.real(ifftn(z_fft))
    start = np.array(shape)
    end = start * 2
    slices = (slice(start[0], end[0]), slice(start[1], end[1]), slice(start[2], end[2]))
    return field[slices]

def gaussian_to_lognormal(field, mean, variance):
    sigma = np.sqrt(np.log(variance / mean**2 + 1))
    mu = np.log(mean) - 0.5 * sigma**2
    return np.exp(field * sigma + mu)

def plot_plotly_3d(field, cutoff):
    nx, ny, nz = field.shape
    spacing = (1, 1, 1)

    mask = field >= cutoff
    coords = np.argwhere(mask)

    if len(coords) == 0:
        print("Aucun voxel au-dessus du seuil cutoff.")
        return

    x = coords[:, 0] * spacing[0]
    y = coords[:, 1] * spacing[1]
    z = coords[:, 2] * spacing[2]

    values = field[mask]

    fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=values,
        colorscale='Viridis',
        cmin=0,      # <-- Ajout ici
        cmax=10,     # <-- Ajout ici
        opacity=0.8,
        colorbar=dict(title='Teneur (ppm)'),
        showscale=True,
        symbol='square'
    )
    ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectratio=dict(x=nx*spacing[0], y=ny*spacing[1], z=nz*spacing[2]),
            camera=dict(
                eye=dict(x=20000, y=20000, z=20000) 
            )
        ),
        title='Simulation 3D d\'un gisement d\'or'
    )
    fig.show()


def interactive_sim_3d(mean, variance, cutoff, ax, ay, az, angle_x, angle_y, angle_z):
    shape = (100, 100, 50)
    field = fftma_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=variance)
    field = gaussian_to_lognormal(field, mean, variance)
    plot_plotly_3d(field, cutoff)

interact(interactive_sim_3d,
         mean=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Moyenne'),
         variance=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Variance'),
         cutoff=FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Teneur coupure'),
         ax=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Ax'),
         ay=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Ay'),
         az=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Az'),
         angle_x=IntSlider(value=0, min=0, max=360, step=1, description='Angle X'),
         angle_y=IntSlider(value=0, min=0, max=360, step=1, description='Angle Y'),
         angle_z=IntSlider(value=0, min=0, max=360, step=1, description='Angle Z'))


interactive(children=(FloatSlider(value=1.0, description='Moyenne', max=10.0, min=0.1), FloatSlider(value=1.0,…

<function __main__.interactive_sim_3d(mean, variance, cutoff, ax, ay, az, angle_x, angle_y, angle_z)>